Given a matrix and a target, return the number of non-empty submatrices that sum to target.

A submatrix x1, y1, x2, y2 is the set of all cells matrix[x][y] with x1 <= x <= x2 and y1 <= y <= y2.

Two submatrices (x1, y1, x2, y2) and (x1', y1', x2', y2') are different if they have some coordinate that is different: for example, if x1 != x1'.

 

Example 1:


Input: matrix = [[0,1,0],[1,1,1],[0,1,0]], target = 0
Output: 4
Explanation: The four 1x1 submatrices that only contain 0.
Example 2:

Input: matrix = [[1,-1],[-1,1]], target = 0
Output: 5
Explanation: The two 1x2 submatrices, plus the two 2x1 submatrices, plus the 2x2 submatrix.
Example 3:

Input: matrix = [[904]], target = 0
Output: 0


1. **Submatrix**: Any rectangle inside the matrix. It can be of size 1x1, 1x2, 2x2, etc.
2. **Sum of a submatrix**: Add all elements inside that rectangle.
3. **Goal**: Count all submatrices whose sum exactly equals `target`.


### **Brute Force Approach**

1. **Idea**: Try **every possible submatrix** and calculate its sum.
2. To generate a submatrix, we need:

   * **Top row** (`r1`) from 0 to `rows-1`
   * **Bottom row** (`r2`) from `r1` to `rows-1`
   * **Left column** (`c1`) from 0 to `cols-1`
   * **Right column** (`c2`) from `c1` to `cols-1`
3. For each `(r1, r2, c1, c2)` combination, calculate the sum of the submatrix and check if it equals `target`.

---

### **Brute Force Pseudocode**

```
count = 0
for r1 in 0..rows-1:
    for r2 in r1..rows-1:
        for c1 in 0..cols-1:
            for c2 in c1..cols-1:
                sum_submatrix = 0
                for i in r1..r2:
                    for j in c1..c2:
                        sum_submatrix += mat[i][j]
                if sum_submatrix == target:
                    count += 1
return count
```

---

### **Time Complexity**

* Generating all submatrices: `O(rows^2 * cols^2)`
* Calculating the sum of each submatrix naively: `O(rows * cols)`
* **Overall**: `O(rows^3 * cols^3)` → very slow for large matrices.

---

### **Next Step for Optimization**

* We can **precompute prefix sums** for the matrix to get any submatrix sum in `O(1)` instead of `O(rows*cols)`.
* Later, we can reduce it further using **1D prefix sum + hash map**, treating each pair of rows as a 1D array (like subarray sum equals target).



In [ ]:
# pre computed the pre-fix matrix.
class Solution:
    def numSubmatrixSumTarget(self, matrix: list[list[int]], target: int) -> int:
        rows, cols = len(matrix), len(matrix[0])
        count = 0

        # Step 1: build 2D prefix sum
        prefix = [[0]*(cols+1) for _ in range(rows+1)]
        for r in range(rows):
            for c in range(cols):
                prefix[r+1][c+1] = matrix[r][c] + prefix[r][c+1] + prefix[r+1][c] - prefix[r][c]

        # Step 2: iterate over all submatrices
        for r1 in range(rows):
            for c1 in range(cols):
                for r2 in range(r1, rows):
                    for c2 in range(c1, cols):
                        # sum of submatrix from (r1,c1) to (r2,c2)
                        sub_sum = prefix[r2+1][c2+1] - prefix[r1][c2+1] - prefix[r2+1][c1] + prefix[r1][c1]
                        if sub_sum == target:
                            count += 1

        return count
# tc - O(r^2 * c^2) + O(r*c) [ for pre-sum calculation]
# sc - O(r * c)


In [ ]:
class Solution:
    def numSubmatrixSumTarget(self, matrix: list[list[int]], target: int) -> int:
        rows, cols = len(matrix), len(matrix[0])
        count = 0

        # Step 1: compute prefix sum for each row
        for r in range(rows):
            for c in range(1, cols):
                matrix[r][c] += matrix[r][c-1]  # NOTE: we are changing the matrix in place
 
        # Step 2: iterate over all pairs of columns
        for c1 in range(cols):
            for c2 in range(c1, cols):
                print(c1,c2)
                sums = {0: 1}  # prefix sum hashmap
                cur_sum = 0
                for r in range(rows):
                    print("row", r)
                    # sum of elements in row r between columns c1 and c2
                    row_sum = matrix[r][c2] - (matrix[r][c1-1] if c1 > 0 else 0)
                    cur_sum += row_sum
                    count += sums.get(cur_sum - target, 0)
                    sums[cur_sum] = sums.get(cur_sum, 0) + 1

        return count


# tc - O( col ^ 2 * row)
# sc - O(rows)

In [5]:
Solution().numSubmatrixSumTarget(matrix = [[0,1,0],[1,1,1],[0,1,0]], target = 0)

0 0
row 0
row 1
row 2
0 1
row 0
row 1
row 2
0 2
row 0
row 1
row 2
1 1
row 0
row 1
row 2
1 2
row 0
row 1
row 2
2 2
row 0
row 1
row 2


4

In [ ]:
✅ Explanation

Row Prefix Sum: matrix[r][c] stores sum of row r from 0 to c.

Column Pair Loop: Iterate over all pairs (c1, c2). For these columns, each row becomes a single number representing the sum between c1 and c2.

1D Subarray Sum HashMap: Treat each pair of columns as a 1D array problem: number of subarrays with sum = target.

Count: Increment count for each subarray that matches the target.

✅ Time Complexity

Prefix sum: O(rows * cols)

Column pairs: O(cols^2 * rows) → each row is processed once for each column pair

Overall: O(rows * cols^2)

✅ Space Complexity

Hash map for each column pair: O(rows)